In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [3]:
import csv
import logging
import argparse
import random
from tqdm import trange, tqdm
import matplotlib as mpl
mpl.use('Agg')

In [4]:
import matplotlib.pyplot as plt

from scipy import interp

from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, auc, confusion_matrix, classification_report
from sklearn.utils.fixes import signature
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from torch import nn

/usr/local/share/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/share/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [5]:
print(torch.__version__)

1.7.1


In [6]:
torch.cuda.is_available(), torch.cuda.device_count()

/usr/local/share/anaconda3/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


(False, 0)

In [7]:
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam
#important
from modeling_readmission import BertForSequenceClassification
print('in the modeling class')

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s', 
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
logger = logging.getLogger(__name__)

in the modeling class
in the modeling class


In [8]:
class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

In [9]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id

In [10]:
class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError() #this is a runtime error

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    # classmethod decoration, doesn't need to sepecify self in the defination
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                lines.append(line)
            return lines
        
    @classmethod
    def _read_csv(cls, input_file):
        """Reads a comma separated value file."""
        file=pd.read_csv(input_file)
        lines=zip(file.ID,file.TEXT,file.Label)
        return lines


In [11]:
import builtins
#help(builtins)

In [12]:
class readmissionProcessor(DataProcessor):
    def get_train_examples(self, data_dir):
        logger.info("LOOKING AT {}".format(os.path.join(data_dir, "train.csv")))
        return self._create_examples(
            self._read_csv(os.path.join(data_dir, "train.csv")), "train")
    
    def get_dev_examples(self, data_dir):
        return self._create_examples(
            self._read_csv(os.path.join(data_dir, "val.csv")), "val")
    
    def get_test_examples(self, data_dir):
        return self._create_examples(
            self._read_csv(os.path.join(data_dir, "test.csv")), "test")
    
    def get_labels(self):
        return ["0", "1"]
        
    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i) 
            text_a = line[1]
            label = str(int(line[2])) 
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples

In [ ]:
! head -1 /data/users/haotiang/20211217_num2word/3days/train.csv

In [ ]:
test_train_path = '/data/users/haotiang/20211217_num2word/3days/train.csv'
for i, line in enumerate(DataProcessor._read_csv(test_train_path)):
    if i < 2:
        print(line)

In [ ]:
test_train_path = '/data/users/haotiang/20211217_num2word/3days/train.csv'
examples = []
for i, line in enumerate(DataProcessor._read_csv(test_train_path)):
    if i < 2:
        guid = "%s-%s" % ("test", i) 
        text_a = line[1] # get text
        label = str(int(line[2])) # get label
        examples.append((guid, text_a, label))
print(examples)

In [ ]:
test_train_path = '/data/users/haotiang/20211217_num2word/3days/train.csv'
examples = []
for i, line in enumerate(DataProcessor._read_csv(test_train_path)):
    guid = "%s-%s" % ("test", i) 
    text_a = line[1] # get text
    label = str(int(line[2])) # get label
    examples.append(
        InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
len(examples)

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i

    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b) # text_b is not useful in our task

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[0:(max_seq_length - 2)] # only trained with 0: 512-2

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = []
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in tokens_a:
            tokens.append(token)
            segment_ids.append(0)
        tokens.append("[SEP]")
        segment_ids.append(0)

        if tokens_b:
            for token in tokens_b:
                tokens.append(token)
                segment_ids.append(1)
            tokens.append("[SEP]")
            segment_ids.append(1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens) 
        # BertTokenizer.from_pretrained('bert-base-uncased').convert_tokens_to_ids
        # convert tokens to number (max 512)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        #print (example.label)
        label_id = label_map[example.label]
        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %d)" % (example.label, label_id))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id))
    return features

01/05/2022 12:14:18 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/haotiang/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [ ]:
label_list

In [ ]:
label_map = {}
for (i, label) in enumerate(label_list):
    label_map[label] = i

In [ ]:
label_map

In [ ]:
! head -1 /data/users/haotiang/20211217_num2word/3days/train.csv

In [ ]:
train_input_file = '/data/users/haotiang/20211217_num2word/3days/train.csv'
pd_train = pd.read_csv(train_input_file)

In [ ]:
pd_train.head(5)

In [ ]:
pd_train.sort_values(by=['ID']).head(5)

In [ ]:
train_examples[0].text_a

In [ ]:
train_examples[0].guid

In [ ]:
# test the first line in train.csv under 3days folder
features = []
trial_example = train_examples[0]
tokens_a = tokenizer.tokenize(trial_example.text_a)
tokens_b = None
if trial_example.text_b:
    tokens_b = tokenizer.tokenize(trial_example.text_b)

In [ ]:
trial_example.text_b

In [ ]:
train_examples[0]

In [ ]:
tokens_a

In [ ]:
features = []
test_train = []
for (ex_index, example) in enumerate(train_examples):
    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    test_train.append((example.guid, tokens_a, tokens_b))

In [ ]:
pd_1 = pd.DataFrame(test_train)
pd_1['len'] = pd_1[1].map(lambda x: len(x))
pd_1.sort_values(by=['len'], ascending=False)

In [ ]:
pd_1[pd_1['len'] < 512].sort_values(by=['len'])

In [ ]:
pd_1[pd_1['len'] < 512]

In [ ]:
features = []
test_train = []
for (ex_index, example) in enumerate(train_examples):
    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    test_train.append((example.guid, tokens_a, tokens_b))

In [22]:
max_seq_length = 512

In [ ]:
test_train[0][1]

In [ ]:
tokenizer.convert_tokens_to_ids??

In [ ]:
tokenizer.convert_tokens_to_ids(test_train[0][1][: 512-2])

In [ ]:
1+1

In [23]:
train_features = convert_examples_to_features(
            train_examples, label_list, max_seq_length, tokenizer)

01/05/2022 12:15:33 - INFO - __main__ -   *** Example ***
01/05/2022 12:15:33 - INFO - __main__ -   guid: train-0
01/05/2022 12:15:33 - INFO - __main__ -   tokens: [CLS] the degree of hem ##or ##rh ##age appears relatively stable since the recent prior examination . areas of basil ##ar consolidation identified as on prior study . ( over ) four : forty - two pm ct ##a abd w & w / o c & rec ##ons ; ct pe ##lvis w / contrast clip # reason : please eva ##l hem ##ang ##iom ##a , liver bleeding , other abd pathology . admitting diagnosis : gi bleed ; tel ##eme ##try field of view : forty contrast : visa ##pa ##que am ##t : one hundred and fifty final report ( con ##t ) swan ##a status d : admitted from ed oriented x ##3 . . c / o abd pain with breathing or moving abd di ##sten ##ded firm . . o ##2 dc ' d by ho . . iv d ##5 ##w w / bi ##car ##b @ 100 ##cc foley with ad ##qua ##te hu ##o r : vs ##s p : will continue to monitor closely for further bleeding . . pc ' s per ho update o : see care 

01/05/2022 12:15:33 - INFO - __main__ -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

01/05/2022 12:15:33 - INFO - __main__ -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

01/05/2022 12:15:33 - INFO - __main__ -   label: 1 (id = 1)
01/05/2022 12:15:33 - INFO - __main__ -   *** Example ***
01/05/2022 12:15:33 - INFO - __main__ -   guid: train-4
01/05/2022 12:15:33 - INFO - __main__ -   tokens: [CLS] sa ##git ##tal and axial , and flair t ##2 and su ##sc ##ept ##ibility as well as diffusion axial images of the brain were obtained . following ga ##do ##lini ##um t ##1 sa ##git ##tal axial and corona ##l images were obtained . comparison was made with the previous mri study of . findings : the diffusion images demonstrate areas of restricted diffusion in the left par ##ie ##tal region slightly posterior to the area of surgical defect . this is suspicious for acute in ##far ##ct in the left par ##ie ##tal region . a follow ##up examination is recommended . post operative changes are seen in the left frontal region with blood products . following ga ##do ##lini ##um subtle marginal enhancement is seen at the surgical bed . no evidence of mid ##line shift mass 

KeyboardInterrupt: 

# Main

## 1. Fine-tune

In [13]:
parser = argparse.ArgumentParser()
parser.add_argument?

1. 

In [14]:
local_rank = -1
train_batch_size = 56 # testing, reference of Kevin's artical
no_cuda = False

In [15]:
processors = {
    "readmission": readmissionProcessor
}
device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu") # defult setting
n_gpu = torch.cuda.device_count() # n_gpu = 0
logger.info("device %s n_gpu %d distributed training %r", device, n_gpu, False)
gradient_accumulation_steps = 1 # default
num_train_epochs = 3 # default, reference of Kevin's artical


random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

task_name = 'readmission'
processor = processors[task_name]()
label_list = processor.get_labels() # ['0', '1']

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_examples = None
num_train_steps = None

train_examples = processor.get_train_examples('/data/users/haotiang/20211217_num2word/3days')
num_train_steps = int(
    len(train_examples) / train_batch_size / gradient_accumulation_steps * num_train_epochs)

01/05/2022 12:12:52 - INFO - __main__ -   device cpu n_gpu 0 distributed training False
01/05/2022 12:12:53 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/haotiang/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
01/05/2022 12:12:53 - INFO - __main__ -   LOOKING AT /data/users/haotiang/20211217_num2word/3days/train.csv


#### end

In [ ]:
torch.cuda.is_available(), torch.cuda.device_count()

In [ ]:
for i in processors:
    print(i)

In [ ]:
processors[task_name]()

In [ ]:
processor = processors[task_name]()
label_list = processor.get_labels()
label_list

In [ ]:
len(train_examples)

In [ ]:
num_train_steps

2 Prepare model and optimizer

In [16]:
bert_model = '/data/users/haotiang/model/pretraining'
model = BertForSequenceClassification.from_pretrained(bert_model, 1) #load pretrained clinicalBERT model
model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
    ]
# Implements BERT version of Adam algorithm with weight decay fix.
learning_rate = 2e-5 # by author
warmup_proportion = 0.1 #default
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=learning_rate,
                     warmup=warmup_proportion,
                     t_total=num_train_steps)

global_step = 0
train_loss=100000
number_training_steps=1
global_step_check=0
#train_loss_history=[]



01/05/2022 12:13:14 - INFO - modeling_readmission -   loading archive file /data/users/haotiang/model/pretraining
01/05/2022 12:13:14 - INFO - modeling_readmission -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



In [21]:
#train_features = convert_examples_to_features(
#    train_examples, label_list, args.max_seq_length, tokenizer)
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_examples))
logger.info("  Batch size = %d", train_batch_size)
logger.info("  Num steps = %d", num_train_steps)

all_input_ids = torch.tensor([f.input_ids for f in train_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in train_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in train_features], dtype=torch.long)
all_label_ids = torch.tensor([f.label_id for f in train_features], dtype=torch.long)
train_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)

if local_rank == -1:
    train_sampler = RandomSampler(train_data) # A Sampler that returns random indices
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=train_batch_size)
# each batch has shape of (4, 56, 512) (only lable_ids has shape 56)

01/05/2022 12:14:43 - INFO - __main__ -   ***** Running training *****
01/05/2022 12:14:43 - INFO - __main__ -     Num examples = 49800
01/05/2022 12:14:43 - INFO - __main__ -     Batch size = 56
01/05/2022 12:14:43 - INFO - __main__ -     Num steps = 2667


NameError: name 'train_features' is not defined

#### end

In [ ]:
for i, v in enumerate(train_features):
    if i < 3:
        print(type(v))

In [ ]:
for i, v in enumerate(train_data):
    if i < 3:
        print(v)

In [ ]:
len(train_sampler)

In [ ]:
for i, v in enumerate(train_sampler):
    if i < 10:
        print(v)

In [ ]:
batch_input_ids, batch_input_mask, batch_segment_ids, batch_label_ids = next(iter(train_dataloader))
print(batch_input_ids.size(), batch_input_mask.size(), batch_segment_ids.size(), batch_label_ids.size())

In [ ]:
! ls /data/users/haotiang/model/pretraining

In [ ]:
BertForSequenceClassification.from_pretrained??

In [ ]:
model

In [ ]:
param_optimizer

In [ ]:
for n, p in param_optimizer:
    if 'gamma' in n:
        print(n)

In [ ]:
optimizer_grouped_parameters

In [ ]:
BertAdam??

In [ ]:
optimizer

3 train the model

In [ ]:
# default setting
num_train_epochs = 3.0
fp16 = False
loss_scale = 128
gradient_accumulation_steps = 1
optimize_on_cpu = False

In [ ]:
train_loss_history=[]
model.train()
# model = BertForSequenceClassification.from_pretrained(bert_model, 1) #load pretrained clinicalBERT model

In [ ]:
for step, batch in enumerate(train_dataloader):
    if step < 3:
        print([t for t in batch])

In [ ]:
# one epoch
train_loss_history=[]
tr_loss = 0
nb_tr_examples, nb_tr_steps = 0, 0
for step, batch in enumerate(train_dataloader):
    if step < 1: # one batch
        batch = tuple(t.to(device) for t in batch)
        input_ids, input_mask, segment_ids, label_ids = batch
        loss, logits = model(input_ids, segment_ids, input_mask, label_ids)
        if n_gpu > 1:
            loss = loss.mean() # mean() to average on multi-gpu.
        if fp16 and loss_scale != 1.0:
            # rescale loss for fp16 training
            # see https://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html
            loss = loss * loss_scale
        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps
        loss.backward()
        train_loss_history.append(loss.item())
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        
        if (step + 1) % gradient_accumulation_steps == 0:
            # below part is escaped
            if fp16 or optimize_on_cpu:
                if fp16 and loss_scale != 1.0:
                    # scale down gradients for fp16 training
                    for param in model.parameters():
                        if param.grad is not None:
                            param.grad.data = param.grad.data / loss_scale
                is_nan = set_optimizer_params_grad(param_optimizer, model.named_parameters(), test_nan=True)
                if is_nan:
                    logger.info("FP16 TRAINING: Nan in gradients, reducing loss scaling")
                    loss_scale = loss_scale / 2
                    model.zero_grad()
                    continue
                optimizer.step()
                copy_optimizer_params_to_model(model.named_parameters(), param_optimizer)
            else:
                # only this is emplementeed
                optimizer.step()
            model.zero_grad()
            global_step += 1

In [ ]:
if (step + 1) % gradient_accumulation_steps == 0:
    if fp16 or optimize_on_cpu:
        if fp16 and loss_scale != 1.0:
            # scale down gradients for fp16 training
            for param in model.parameters():
                if param.grad is not None:
                    print(1)
        print(2)
        #is_nan = set_optimizer_params_grad(param_optimizer, model.named_parameters(), test_nan=True)

In [ ]:
train_loss_history, tr_loss, nb_tr_examples, nb_tr_steps

In [ ]:
loss, logits

In [ ]:
loss

## 2. eval

In [24]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def convert_examples_to_features(examples, label_list, max_seq_length, tokenizer):
    """Loads a data file into a list of `InputBatch`s."""

    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i

    features = []
    for (ex_index, example) in enumerate(examples):
        tokens_a = tokenizer.tokenize(example.text_a)

        tokens_b = None
        if example.text_b:
            tokens_b = tokenizer.tokenize(example.text_b) # text_b is not useful in our task

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > max_seq_length - 2:
                tokens_a = tokens_a[0:(max_seq_length - 2)] # only trained with 0: 512-2

        # The convention in BERT is:
        # (a) For sequence pairs:
        #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
        #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
        # (b) For single sequences:
        #  tokens:   [CLS] the dog is hairy . [SEP]
        #  type_ids: 0   0   0   0  0     0 0
        #
        # Where "type_ids" are used to indicate whether this is the first
        # sequence or the second sequence. The embedding vectors for `type=0` and
        # `type=1` were learned during pre-training and are added to the wordpiece
        # embedding vector (and position vector). This is not *strictly* necessary
        # since the [SEP] token unambigiously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.
        #
        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        tokens = []
        segment_ids = []
        tokens.append("[CLS]")
        segment_ids.append(0)
        for token in tokens_a:
            tokens.append(token)
            segment_ids.append(0)
        tokens.append("[SEP]")
        segment_ids.append(0)

        if tokens_b:
            for token in tokens_b:
                tokens.append(token)
                segment_ids.append(1)
            tokens.append("[SEP]")
            segment_ids.append(1)

        input_ids = tokenizer.convert_tokens_to_ids(tokens) 
        # BertTokenizer.from_pretrained('bert-base-uncased').convert_tokens_to_ids
        # convert tokens to number (max 512)

        # The mask has 1 for real tokens and 0 for padding tokens. Only real
        # tokens are attended to.
        input_mask = [1] * len(input_ids)

        # Zero-pad up to the sequence length.
        while len(input_ids) < max_seq_length:
            input_ids.append(0)
            input_mask.append(0)
            segment_ids.append(0)

        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length
        #print (example.label)
        label_id = label_map[example.label]
        if ex_index < 5:
            logger.info("*** Example ***")
            logger.info("guid: %s" % (example.guid))
            logger.info("tokens: %s" % " ".join(
                    [str(x) for x in tokens]))
            logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
            logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
            logger.info(
                    "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
            logger.info("label: %s (id = %d)" % (example.label, label_id))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id))
    return features

01/05/2022 12:20:58 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/haotiang/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [25]:
do_eval = True
data_dir = '/data/users/haotiang/20211217_num2word/3days/'
eval_batch_size = 2 # default
max_seq_length = 512 # from the artical
local_rank = -1

In [26]:
! ls /data/users/haotiang/

20211217_num2word      data.zip  pretrained_model.zip  result_early
20211217_num2word.zip  __MACOSX  result_discharge      result_early_finetuned
data		       model	 result_discharge_num


In [27]:
processors = {
    "readmission": readmissionProcessor
}
device = torch.device("cuda" if torch.cuda.is_available() and not no_cuda else "cpu") # defult setting
n_gpu = torch.cuda.device_count() # n_gpu = 0
logger.info("device %s n_gpu %d distributed training %r", device, n_gpu, False)
gradient_accumulation_steps = 1 # default
num_train_epochs = 3 # default, reference of Kevin's artical


random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

task_name = 'readmission'
processor = processors[task_name]()
label_list = processor.get_labels() # ['0', '1']

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

01/05/2022 12:21:32 - INFO - __main__ -   device cpu n_gpu 0 distributed training False
01/05/2022 12:21:32 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/haotiang/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [28]:
label_list

['0', '1']

In [29]:
tokenizer

In [30]:
local_rank, device

(-1, device(type='cpu'))

step 1

In [31]:
m = nn.Sigmoid()
if do_eval:
    eval_examples = processor.get_test_examples(data_dir)
    eval_features = convert_examples_to_features(
        eval_examples, label_list, max_seq_length, tokenizer)
    logger.info("***** Running evaluation *****")
    logger.info("  Num examples = %d", len(eval_examples))
    logger.info("  Batch size = %d", eval_batch_size)
    all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
    all_label_ids = torch.tensor([f.label_id for f in eval_features], dtype=torch.long)
    eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    if local_rank == -1:
        eval_sampler = SequentialSampler(eval_data) # shuffle the test data
    else:
        eval_sampler = DistributedSampler(eval_data)
    eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=eval_batch_size)

01/05/2022 12:22:25 - INFO - __main__ -   *** Example ***
01/05/2022 12:22:25 - INFO - __main__ -   guid: test-0
01/05/2022 12:22:25 - INFO - __main__ -   tokens: [CLS] sin ##us ta ##chy ##card ##ia . generalized low voltage . delayed r wave progression with late pre ##cor ##dial q ##rs transition . findings are non - specific . clinical correlation is suggested . since the previous tracing of same date sin ##us ta ##chy ##card ##ia rate is faster . tracing # one sin ##us rhythm . delayed r wave progression with late pre ##cor ##dial q ##rs transition . generalized low q ##rs voltage . findings are non - specific . clinical correlation is suggested . since the previous tracing of the rate is faster and voltage is lower . title : forty - five y / o man with h ##x of et ##oh ci ##rr ##hosis who arrived for planned liver transplant . went to or am and received pt from or approx one thousand , eight hundred int ##uba ##ted and se ##date ##d on prop ##of ##ol . transplant , liver assessment

01/05/2022 12:22:25 - INFO - __main__ -   input_ids: 101 3653 25918 8082 3152 2005 11290 22291 1012 6028 1024 6643 1998 11457 3108 2557 27341 1012 7831 1024 4102 2000 2557 14413 2013 1012 9556 1024 2540 2946 2003 17844 11792 1010 2714 2000 3188 2817 1012 2045 2003 10256 21908 21449 25707 1012 2045 2003 6540 2235 2187 20228 11236 2389 1041 4246 14499 1012 13483 8208 1997 2157 27263 2278 2240 1012 8605 1024 4003 18994 29107 2135 1012 2053 18583 1012 2235 2187 20228 11236 2389 1041 4246 14499 1012 5408 1024 3174 1011 2416 7610 3108 1006 12109 9706 1007 1025 1011 10920 1011 2416 2011 2168 7522 1001 3114 1024 1055 1013 1052 19330 2102 1012 9345 2140 2005 7859 1010 2058 11066 17927 11616 1024 11290 19067 2966 4650 1024 5659 1011 2274 2095 2214 2158 4078 4017 18924 2015 1010 20014 19761 3064 3114 2005 2023 7749 1024 1055 1013 1052 19330 2102 1012 9345 2140 2005 7859 1010 2058 11066 2345 3189 9706 3108 5408 1024 4228 1011 2274 1052 1012 1049 1012 1010 2381 1024 4078 4017 18924 1010 2044 11290 

01/05/2022 12:22:25 - INFO - __main__ -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

01/05/2022 12:22:25 - INFO - __main__ -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

01/05/2022 12:22:25 - INFO - __main__ -   label: 1 (id = 1)
01/05/2022 12:22:55 - INFO - __main__ -   ***** Running evaluation *****
01/05/2022 12:22:55 - INFO - __main__ -     Num examples = 5687
01/05/2022 12:22:55 - INFO - __main__ -     Batch size = 2


end

In [32]:
eval_examples = processor.get_test_examples(data_dir)

In [33]:
len(eval_examples)

5687

In [34]:
pd_2 = pd.read_csv('/data/users/haotiang/20211217_num2word/3days/test.csv')
#pd_2['len'] = pd_2[].map(lambda x: len(x))
#pd_1.sort_values(by=['len'], ascending=False)
pd_2

,Unnamed: 0,ID,TEXT,Label
0,536,100133.0,sinus tachycardia. generalized low voltage. de...,1.0
1,537,100133.0,preoperative films for liver transplant. techn...,1.0
2,538,100133.0,tip of the left transjugular swan-ganz cathete...,1.0
3,539,100133.0,transplant/sicu teams aware of huo or lack the...,1.0
4,540,100133.0,hct twenty-six.two% with four hundred and thir...,1.0
...,...,...,...,...
5682,401376,199930.0,for obstruction. nursing progress note full co...,0.0
5683,401377,199930.0,also displayed. ct of the chest with and witho...,0.0
5684,401378,199930.0,abdomen. ct of the pelvis with and without iv ...,0.0
5685,401379,199930.0,totally drained 3800cc. later around two thous...,0.0


In [35]:
features = []
test_eval = []
for (ex_index, example) in enumerate(eval_examples):
    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)

    test_eval.append((example.guid, tokens_a, tokens_b))
    
pd_3 = pd.DataFrame(test_eval)
pd_3['len'] = pd_3[1].map(lambda x: len(x))
#pd_1.sort_values(by=['len'], ascending=False)

In [36]:
pd_3

,0,1,2,len
0,test-0,"[sin, ##us, ta, ##chy, ##card, ##ia, ., genera...",None,521
1,test-1,"[pre, ##oper, ##ative, films, for, liver, tran...",None,544
2,test-2,"[tip, of, the, left, trans, ##ju, ##gul, ##ar,...",None,516
3,test-3,"[transplant, /, sic, ##u, teams, aware, of, hu...",None,533
4,test-4,"[hc, ##t, twenty, -, six, ., two, %, with, fou...",None,591
...,...,...,...,...
5682,test-5682,"[for, obstruction, ., nursing, progress, note,...",None,579
5683,test-5683,"[also, displayed, ., ct, of, the, chest, with,...",None,554
5684,test-5684,"[abdomen, ., ct, of, the, pe, ##lvis, with, an...",None,492
5685,test-5685,"[totally, drained, 380, ##0, ##cc, ., later, a...",None,601


In [ ]:
pd_3.sort_values(['len'])

In [37]:
! wc -l '/data/users/haotiang/20211217_num2word/3days/test.csv'

5688 /data/users/haotiang/20211217_num2word/3days/test.csv


In [39]:
label_map = {}
for (i, label) in enumerate(label_list):
    label_map[label] = i

features = []
for (ex_index, example) in enumerate(eval_examples):
    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b) # text_b is not useful in our task

    if tokens_b:
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[0:(max_seq_length - 2)] # only trained with 0: 512-2

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids: 0   0  0    0    0     0       0 0    1  1  1  1   1 1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids: 0   0   0   0  0     0 0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambigiously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = []
    segment_ids = []
    tokens.append("[CLS]")
    segment_ids.append(0)
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(0)
    tokens.append("[SEP]")
    segment_ids.append(0)

    if tokens_b:
        for token in tokens_b:
            tokens.append(token)
            segment_ids.append(1)
        tokens.append("[SEP]")
        segment_ids.append(1)

    input_ids = tokenizer.convert_tokens_to_ids(tokens) 
    # BertTokenizer.from_pretrained('bert-base-uncased').convert_tokens_to_ids
    # convert tokens to number (max 512)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1] * len(input_ids)

    # Zero-pad up to the sequence length.
    while len(input_ids) < max_seq_length:
        input_ids.append(0)
        input_mask.append(0)
        segment_ids.append(0)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length
    #print (example.label)
    label_id = label_map[example.label]
    if ex_index < 5:
        logger.info("*** Example ***")
        logger.info("guid: %s" % (example.guid))
        logger.info("tokens: %s" % " ".join(
                [str(x) for x in tokens]))
        logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
        logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
        logger.info(
                "segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
        logger.info("label: %s (id = %d)" % (example.label, label_id))

    features.append(
            InputFeatures(input_ids=input_ids,
                          input_mask=input_mask,
                          segment_ids=segment_ids,
                          label_id=label_id))

01/05/2022 12:25:06 - INFO - __main__ -   *** Example ***
01/05/2022 12:25:06 - INFO - __main__ -   guid: test-0
01/05/2022 12:25:06 - INFO - __main__ -   tokens: [CLS] sin ##us ta ##chy ##card ##ia . generalized low voltage . delayed r wave progression with late pre ##cor ##dial q ##rs transition . findings are non - specific . clinical correlation is suggested . since the previous tracing of same date sin ##us ta ##chy ##card ##ia rate is faster . tracing # one sin ##us rhythm . delayed r wave progression with late pre ##cor ##dial q ##rs transition . generalized low q ##rs voltage . findings are non - specific . clinical correlation is suggested . since the previous tracing of the rate is faster and voltage is lower . title : forty - five y / o man with h ##x of et ##oh ci ##rr ##hosis who arrived for planned liver transplant . went to or am and received pt from or approx one thousand , eight hundred int ##uba ##ted and se ##date ##d on prop ##of ##ol . transplant , liver assessment

01/05/2022 12:25:06 - INFO - __main__ -   input_ids: 101 3653 25918 8082 3152 2005 11290 22291 1012 6028 1024 6643 1998 11457 3108 2557 27341 1012 7831 1024 4102 2000 2557 14413 2013 1012 9556 1024 2540 2946 2003 17844 11792 1010 2714 2000 3188 2817 1012 2045 2003 10256 21908 21449 25707 1012 2045 2003 6540 2235 2187 20228 11236 2389 1041 4246 14499 1012 13483 8208 1997 2157 27263 2278 2240 1012 8605 1024 4003 18994 29107 2135 1012 2053 18583 1012 2235 2187 20228 11236 2389 1041 4246 14499 1012 5408 1024 3174 1011 2416 7610 3108 1006 12109 9706 1007 1025 1011 10920 1011 2416 2011 2168 7522 1001 3114 1024 1055 1013 1052 19330 2102 1012 9345 2140 2005 7859 1010 2058 11066 17927 11616 1024 11290 19067 2966 4650 1024 5659 1011 2274 2095 2214 2158 4078 4017 18924 2015 1010 20014 19761 3064 3114 2005 2023 7749 1024 1055 1013 1052 19330 2102 1012 9345 2140 2005 7859 1010 2058 11066 2345 3189 9706 3108 5408 1024 4228 1011 2274 1052 1012 1049 1012 1010 2381 1024 4078 4017 18924 1010 2044 11290 

01/05/2022 12:25:06 - INFO - __main__ -   input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

01/05/2022 12:25:06 - INFO - __main__ -   segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

01/05/2022 12:25:06 - INFO - __main__ -   label: 1 (id = 1)


In [40]:
for i, v in enumerate(features):
    if i < 1:
        print(v.input_ids, v.input_mask, v.segment_ids, v.label_id)

[101, 8254, 2271, 11937, 11714, 11522, 2401, 1012, 18960, 2659, 10004, 1012, 8394, 1054, 4400, 14967, 2007, 2397, 3653, 27108, 27184, 1053, 2869, 6653, 1012, 9556, 2024, 2512, 1011, 3563, 1012, 6612, 16902, 2003, 4081, 1012, 2144, 1996, 3025, 16907, 1997, 2168, 3058, 8254, 2271, 11937, 11714, 11522, 2401, 3446, 2003, 5514, 1012, 16907, 1001, 2028, 8254, 2271, 6348, 1012, 8394, 1054, 4400, 14967, 2007, 2397, 3653, 27108, 27184, 1053, 2869, 6653, 1012, 18960, 2659, 1053, 2869, 10004, 1012, 9556, 2024, 2512, 1011, 3563, 1012, 6612, 16902, 2003, 4081, 1012, 2144, 1996, 3025, 16907, 1997, 1996, 3446, 2003, 5514, 1998, 10004, 2003, 2896, 1012, 2516, 1024, 5659, 1011, 2274, 1061, 1013, 1051, 2158, 2007, 1044, 2595, 1997, 3802, 11631, 25022, 12171, 25229, 2040, 3369, 2005, 3740, 11290, 22291, 1012, 2253, 2000, 2030, 2572, 1998, 2363, 13866, 2013, 2030, 22480, 2028, 4595, 1010, 2809, 3634, 20014, 19761, 3064, 1998, 7367, 13701, 2094, 2006, 17678, 11253, 4747, 1012, 22291, 1010, 11290, 7667, 102

In [41]:
eval_dataloader

In [42]:
batch_input_ids, batch_input_mask, batch_segment_ids, batch_label_ids = next(iter(eval_dataloader))
print(batch_input_ids.size(), batch_input_mask.size(), batch_segment_ids.size(), batch_label_ids.size())

torch.Size([2, 512]) torch.Size([2, 512]) torch.Size([2, 512]) torch.Size([2])


step 2

In [43]:
! ls /data/users/haotiang/model/early_readmission

bert_config.json  pytorch_model.bin


In [63]:
bert_model = '/data/users/haotiang/model/early_readmission'
model = BertForSequenceClassification.from_pretrained(bert_model, 1) #load pretrained clinicalBERT model
model.to(device)
model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
true_labels=[]
pred_labels=[]
logits_history=[]
for input_ids, input_mask, segment_ids, label_ids in tqdm(eval_dataloader):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)
    with torch.no_grad():
        tmp_eval_loss, temp_logits = model(input_ids, segment_ids, input_mask, label_ids)
        logits = model(input_ids,segment_ids,input_mask)

    logits = torch.squeeze(m(logits)).detach().cpu().numpy()
    label_ids = label_ids.to('cpu').numpy()

    outputs = np.asarray([1 if i else 0 for i in (logits.flatten()>=0.5)])
    tmp_eval_accuracy=np.sum(outputs == label_ids)

    true_labels = true_labels + label_ids.flatten().tolist()
    pred_labels = pred_labels + outputs.flatten().tolist()
    logits_history = logits_history + logits.flatten().tolist()

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += input_ids.size(0)
    nb_eval_steps += 1

eval_loss = eval_loss / nb_eval_steps
eval_accuracy = eval_accuracy / nb_eval_examples
df = pd.DataFrame({'logits':logits_history, 'pred_label': pred_labels, 'label':true_labels})

01/05/2022 12:51:58 - INFO - modeling_readmission -   loading archive file /data/users/haotiang/model/early_readmission
01/05/2022 12:51:58 - INFO - modeling_readmission -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

100%|██████████| 2844/2844 [29:28<00:00,  1.61it/s]


In [99]:
bert_model = '/data/users/haotiang/model/early_readmission'
model = BertForSequenceClassification.from_pretrained(bert_model, 2) #load pretrained clinicalBERT model
model.to(device)
model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
true_labels=[]
pred_labels=[]
logits_history=[]
for input_ids, input_mask, segment_ids, label_ids in tqdm(eval_dataloader):
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    label_ids = label_ids.to(device)
    with torch.no_grad():
        tmp_eval_loss, temp_logits = model(input_ids, segment_ids, input_mask, label_ids)
        logits = model(input_ids,segment_ids,input_mask)

    logits = torch.squeeze(m(logits)).detach().cpu().numpy()
    label_ids = label_ids.to('cpu').numpy()

    outputs = np.asarray([1 if i else 0 for i in (logits.flatten()>=0.5)])
    tmp_eval_accuracy=np.sum(outputs == label_ids)

    true_labels = true_labels + label_ids.flatten().tolist()
    pred_labels = pred_labels + outputs.flatten().tolist()
    logits_history = logits_history + logits.flatten().tolist()

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += input_ids.size(0)
    nb_eval_steps += 1

eval_loss = eval_loss / nb_eval_steps
eval_accuracy = eval_accuracy / nb_eval_examples
df = pd.DataFrame({'logits':logits_history, 'pred_label': pred_labels, 'label':true_labels})

01/05/2022 15:28:21 - INFO - modeling_readmission -   loading archive file /data/users/haotiang/model/early_readmission
01/05/2022 15:28:21 - INFO - modeling_readmission -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

  0%|          | 0/2844 [00:00<?, ?it/s]


ValueError: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([4, 1])) is deprecated. Please ensure they have the same size.

In [96]:
BertForSequenceClassification.from_pretrained??
# last layer needs to be trained, freeze the bert, to train the linear model. Here we are using random linear

In [ ]:
BertForSequenceClassification

In [64]:
df

,logits,pred_label,label
0,0.508293,1,1
1,0.700661,1,1
2,0.661133,1,1
3,0.649345,1,1
4,0.986051,1,1
...,...,...,...
5682,0.297631,0,0
5683,0.543576,1,0
5684,0.511333,1,0
5685,0.489785,0,0


In [97]:
do_train

NameError: name 'do_train' is not defined

#### end

In [45]:
for i, v in enumerate(eval_data):
    if i == 0: # only check with the first row
        print(v)

(tensor([  101,  8254,  2271, 11937, 11714, 11522,  2401,  1012, 18960,  2659,
        10004,  1012,  8394,  1054,  4400, 14967,  2007,  2397,  3653, 27108,
        27184,  1053,  2869,  6653,  1012,  9556,  2024,  2512,  1011,  3563,
         1012,  6612, 16902,  2003,  4081,  1012,  2144,  1996,  3025, 16907,
         1997,  2168,  3058,  8254,  2271, 11937, 11714, 11522,  2401,  3446,
         2003,  5514,  1012, 16907,  1001,  2028,  8254,  2271,  6348,  1012,
         8394,  1054,  4400, 14967,  2007,  2397,  3653, 27108, 27184,  1053,
         2869,  6653,  1012, 18960,  2659,  1053,  2869, 10004,  1012,  9556,
         2024,  2512,  1011,  3563,  1012,  6612, 16902,  2003,  4081,  1012,
         2144,  1996,  3025, 16907,  1997,  1996,  3446,  2003,  5514,  1998,
        10004,  2003,  2896,  1012,  2516,  1024,  5659,  1011,  2274,  1061,
         1013,  1051,  2158,  2007,  1044,  2595,  1997,  3802, 11631, 25022,
        12171, 25229,  2040,  3369,  2005,  3740, 11290, 22291,

In [46]:
for i, v in enumerate(eval_data):
    if i < 1: # only check with the first row
        input_ids, input_mask, segment_ids, label_ids = v
        print(input_ids.size(), input_mask.size(), segment_ids.size(), label_ids.size())

torch.Size([512]) torch.Size([512]) torch.Size([512]) torch.Size([])


In [47]:
model.eval()
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
true_labels=[]
pred_labels=[]
logits_history=[]

for i, v in enumerate(eval_dataloader):
    if i < 1: # only check with the first row, repete twice for each row
        input_ids, input_mask, segment_ids, label_ids = v
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)
        with torch.no_grad():
            tmp_eval_loss, temp_logits = model(input_ids, segment_ids, input_mask, label_ids)
            logits = model(input_ids,segment_ids,input_mask)
        logits = torch.squeeze(m(logits)).detach().cpu().numpy()
        label_ids = label_ids.to('cpu').numpy()
        outputs = np.asarray([1 if i else 0 for i in (logits.flatten()>=0.5)]) # key
        tmp_eval_accuracy=np.sum(outputs == label_ids) # 2 numbers are added (0,0) (0,1) (1,0) (1, 1)
        
        true_labels = true_labels + label_ids.flatten().tolist()
        pred_labels = pred_labels + outputs.flatten().tolist()
        logits_history = logits_history + logits.flatten().tolist()
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy # sum of accuracy from batch size 2

        nb_eval_examples += input_ids.size(0) # add 2 each loop
        nb_eval_steps += 1 # add 1 each loop

In [48]:
input_ids

tensor([[  101,  8254,  2271,  ...,  1011,  2274,   102],
        [  101,  3653, 25918,  ...,  1013,  2030,   102]])

In [49]:
tmp_eval_loss

tensor(0.5162)

In [50]:
temp_logits

tensor([[0.0332],
        [0.8504]])

In [51]:
logits

array([0.5082934 , 0.70066136], dtype=float32)

In [53]:
#torch.squeeze(m(logits)).detach().cpu().numpy()

In [54]:
logits

array([0.5082934 , 0.70066136], dtype=float32)

In [55]:
label_ids

array([1, 1])

In [56]:
np.asarray([1 if i else 0 for i in (logits.flatten()>=0.5)])

array([1, 1])

In [57]:
tmp_eval_accuracy

2

In [58]:
true_labels, pred_labels, logits_history

([1, 1], [1, 1], [0.5082933902740479, 0.7006613612174988])

In [59]:
tmp_eval_loss

tensor(0.5162)

In [60]:
tmp_eval_loss.mean().item()

0.5162135362625122

In [61]:
input_ids.size(0)

2

In [65]:
! pwd

/home/haotiang/notebook


step 2.2

![title](img/Algorithm_of_Scalable_Readmission.png)

In [70]:
score = logits_history

df_test = pd.read_csv(os.path.join(data_dir, "test.csv"))
df_test['pred_score'] = score
df_sort = df_test.sort_values(by=['ID'])

#score 
# algorithm is based on above 
# c is set to 2 as eval_batch_size = 2 # default
temp = (df_sort.groupby(['ID'])['pred_score'].agg(max)+df_sort.groupby(['ID'])['pred_score'].agg(sum)/2)/(1+df_sort.groupby(['ID'])['pred_score'].agg(len)/2)
x = df_sort.groupby(['ID'])['Label'].agg(np.min).values # although min is used here, id is actually the same
df_out = pd.DataFrame({'logits': temp.values, 'ID': x})

fpr, tpr, thresholds = roc_curve(x, temp.values)
auc_score = auc(fpr, tpr)

plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr, tpr, label='Val (area = {:.3f})'.format(auc_score))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
#string = 'auroc_clinicalbert_'+readmission_mode+'.png'
#plt.savefig(os.path.join(output_dir, string))

In [71]:
def vote_score(df, score):
    df['pred_score'] = score
    df_sort = df.sort_values(by=['ID'])
    #score 
    temp = (df_sort.groupby(['ID'])['pred_score'].agg(max)+df_sort.groupby(['ID'])['pred_score'].agg(sum)/2)/(1+df_sort.groupby(['ID'])['pred_score'].agg(len)/2)
    x = df_sort.groupby(['ID'])['Label'].agg(np.min).values
    df_out = pd.DataFrame({'logits': temp.values, 'ID': x})

    fpr, tpr, thresholds = roc_curve(x, temp.values)
    auc_score = auc(fpr, tpr)

    plt.figure(1)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.plot(fpr, tpr, label='Val (area = {:.3f})'.format(auc_score))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve')
    plt.legend(loc='best')
    plt.show()
    string = 'auroc_clinicalbert_'+readmission_mode+'.png'
    plt.savefig(os.path.join(output_dir, string))

    return fpr, tpr, df_out

### end

In [75]:
output_dir

'/home/haotiang/notebook'

In [76]:
score = logits_history

df_test = pd.read_csv(os.path.join(data_dir, "test.csv"))
df_test['pred_score'] = score

In [77]:
df_test

,Unnamed: 0,ID,TEXT,Label,pred_score
0,536,100133.0,sinus tachycardia. generalized low voltage. de...,1.0,0.508293
1,537,100133.0,preoperative films for liver transplant. techn...,1.0,0.700661
2,538,100133.0,tip of the left transjugular swan-ganz cathete...,1.0,0.661133
3,539,100133.0,transplant/sicu teams aware of huo or lack the...,1.0,0.649345
4,540,100133.0,hct twenty-six.two% with four hundred and thir...,1.0,0.986051
...,...,...,...,...,...
5682,401376,199930.0,for obstruction. nursing progress note full co...,0.0,0.297631
5683,401377,199930.0,also displayed. ct of the chest with and witho...,0.0,0.543576
5684,401378,199930.0,abdomen. ct of the pelvis with and without iv ...,0.0,0.511333
5685,401379,199930.0,totally drained 3800cc. later around two thous...,0.0,0.489785


In [78]:
df_sort

,Unnamed: 0,ID,TEXT,Label,pred_score
0,536,100133.0,sinus tachycardia. generalized low voltage. de...,1.0,0.508293
22,558,100133.0,acute distress cardiovascular: (rhythm: regula...,1.0,0.506082
23,559,100133.0,care nutrition: glycemic control: insulin infu...,1.0,0.579559
24,560,100133.0,right portal vein which may be technical in na...,1.0,0.615552
25,561,100133.0,a central line change. study: supine portable ...,1.0,0.923494
...,...,...,...,...,...
5685,401379,199930.0,totally drained 3800cc. later around two thous...,0.0,0.489785
5680,401374,199930.0,impression: ap chest reviewed in the absence o...,0.0,0.308584
5679,401373,199930.0,baseline artifact. sinus rhythm with borderlin...,0.0,0.523618
5681,401375,199930.0,"bowel, and left colon. sma arteriograms demons...",0.0,0.321223


In [79]:
temp = (df_sort.groupby(['ID'])['pred_score'].agg(max)+df_sort.groupby(['ID'])['pred_score'].agg(sum)/2)/(1+df_sort.groupby(['ID'])['pred_score'].agg(len)/2)

In [80]:
temp

ID
100133.0    0.715474
100143.0    0.270472
100294.0    0.645512
100380.0    0.561849
100485.0    0.319974
              ...   
199362.0    0.443447
199452.0    0.433622
199616.0    0.576081
199628.0    0.575013
199930.0    0.441238
Name: pred_score, Length: 518, dtype: float64

In [81]:
df_sort.groupby(['ID'])['Label'].agg(np.min)

ID
100133.0    1.0
100143.0    0.0
100294.0    0.0
100380.0    1.0
100485.0    1.0
           ... 
199362.0    1.0
199452.0    0.0
199616.0    1.0
199628.0    1.0
199930.0    0.0
Name: Label, Length: 518, dtype: float64

In [82]:
x = df_sort.groupby(['ID'])['Label'].agg(np.min).values # although min is used here, id is actually the same
df_out = pd.DataFrame({'logits': temp.values, 'ID': x})

In [83]:
df_out

,logits,ID
0,0.715474,1.0
1,0.270472,0.0
2,0.645512,0.0
3,0.561849,1.0
4,0.319974,1.0
...,...,...
513,0.443447,1.0
514,0.433622,0.0
515,0.576081,1.0
516,0.575013,1.0


In [84]:
roc_curve(x, temp.values)

(array([0.        , 0.        , 0.00392157, 0.00392157, 0.00784314,
        0.00784314, 0.01176471, 0.01176471, 0.01568627, 0.01568627,
        0.01960784, 0.01960784, 0.02352941, 0.02352941, 0.02745098,
        0.02745098, 0.03137255, 0.03137255, 0.03921569, 0.03921569,
        0.04313725, 0.04313725, 0.04705882, 0.04705882, 0.05098039,
        0.05098039, 0.05882353, 0.05882353, 0.06666667, 0.06666667,
        0.0745098 , 0.0745098 , 0.07843137, 0.07843137, 0.08627451,
        0.08627451, 0.09411765, 0.09411765, 0.09803922, 0.09803922,
        0.10196078, 0.10196078, 0.10588235, 0.10588235, 0.10980392,
        0.10980392, 0.11372549, 0.11372549, 0.11764706, 0.11764706,
        0.12156863, 0.12156863, 0.1254902 , 0.1254902 , 0.12941176,
        0.12941176, 0.13333333, 0.13333333, 0.1372549 , 0.1372549 ,
        0.14509804, 0.14509804, 0.14901961, 0.14901961, 0.15294118,
        0.15294118, 0.15686275, 0.15686275, 0.16470588, 0.16470588,
        0.16862745, 0.16862745, 0.17254902, 0.17

step 2.3

In [85]:
def pr_curve_plot(y, y_score):
    precision, recall, _ = precision_recall_curve(y, y_score)
    area = auc(recall,precision)
    step_kwargs = ({'step': 'post'}
                   if 'step' in signature(plt.fill_between).parameters
                   else {})
    
    plt.figure(2)
    plt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall, precision, alpha=0.2, color='b', **step_kwargs)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('Precision-Recall curve: AUC={0:0.2f}'.format(
              area))
    
    string = 'auprc_clinicalbert_'+readmission_mode+'.png'

    plt.savefig(os.path.join(output_dir, string))

In [86]:
score = logits_history
df_test['pred_score'] = score
df_sort = df_test.sort_values(by=['ID'])
#score 
temp = (df_sort.groupby(['ID'])['pred_score'].agg(max)+df_sort.groupby(['ID'])['pred_score'].agg(sum)/2)/(1+df_sort.groupby(['ID'])['pred_score'].agg(len)/2)
y = df_sort.groupby(['ID'])['Label'].agg(np.min).values

precision, recall, thres = precision_recall_curve(y, temp)
pr_thres = pd.DataFrame(data =  list(zip(precision, recall, thres)), columns = ['prec','recall','thres'])
vote_df = pd.DataFrame(data =  list(zip(temp, y)), columns = ['score','label'])

pr_curve_plot(y, temp)

temp = pr_thres[pr_thres.prec > 0.799999].reset_index()

rp80 = 0
if temp.size == 0:
    print('Test Sample too small or RP80=0')
else:
    rp80 = temp.iloc[0].recall
    print('Recall at Precision of 80 is {}', rp80)

Recall at Precision of 80 is {} 0.5855513307984791


In [87]:
def vote_pr_curve(df, score):
    df['pred_score'] = score
    df_sort = df.sort_values(by=['ID'])
    #score 
    temp = (df_sort.groupby(['ID'])['pred_score'].agg(max)+df_sort.groupby(['ID'])['pred_score'].agg(sum)/2)/(1+df_sort.groupby(['ID'])['pred_score'].agg(len)/2)
    y = df_sort.groupby(['ID'])['Label'].agg(np.min).values
    
    precision, recall, thres = precision_recall_curve(y, temp)
    pr_thres = pd.DataFrame(data =  list(zip(precision, recall, thres)), columns = ['prec','recall','thres'])
    vote_df = pd.DataFrame(data =  list(zip(temp, y)), columns = ['score','label'])
    
    pr_curve_plot(y, temp)
    
    temp = pr_thres[pr_thres.prec > 0.799999].reset_index()
    
    rp80 = 0
    if temp.size == 0:
        print('Test Sample too small or RP80=0')
    else:
        rp80 = temp.iloc[0].recall
        print('Recall at Precision of 80 is {}', rp80)

    return rp80

### end

In [88]:
logits_history

[0.5082933902740479,
 0.7006613612174988,
 0.6611330509185791,
 0.6493451595306396,
 0.9860514402389526,
 0.6569519639015198,
 0.4432774782180786,
 0.4770866632461548,
 0.6866414546966553,
 0.6759017109870911,
 0.6439549326896667,
 0.6713142991065979,
 0.5895968079566956,
 0.6126990914344788,
 0.9767186641693115,
 0.536064863204956,
 0.5978657603263855,
 0.5889393091201782,
 0.5681677460670471,
 0.9861887097358704,
 0.6569519639015198,
 0.4539942443370819,
 0.5060821771621704,
 0.5795586109161377,
 0.6155515909194946,
 0.9234936237335205,
 0.6154227256774902,
 0.9573972821235657,
 0.9049018025398254,
 0.9058811664581299,
 0.6706217527389526,
 0.8597192168235779,
 0.8453426361083984,
 0.6923359036445618,
 0.8799079656600952,
 0.8943763375282288,
 0.9410731792449951,
 0.6368076205253601,
 0.6058018207550049,
 0.26662394404411316,
 0.3256610929965973,
 0.14961589872837067,
 0.19679449498653412,
 0.3032858967781067,
 0.6527687907218933,
 0.6237432360649109,
 0.5560782551765442,
 0.53799927

In [89]:
df_test

,Unnamed: 0,ID,TEXT,Label,pred_score
0,536,100133.0,sinus tachycardia. generalized low voltage. de...,1.0,0.508293
1,537,100133.0,preoperative films for liver transplant. techn...,1.0,0.700661
2,538,100133.0,tip of the left transjugular swan-ganz cathete...,1.0,0.661133
3,539,100133.0,transplant/sicu teams aware of huo or lack the...,1.0,0.649345
4,540,100133.0,hct twenty-six.two% with four hundred and thir...,1.0,0.986051
...,...,...,...,...,...
5682,401376,199930.0,for obstruction. nursing progress note full co...,0.0,0.297631
5683,401377,199930.0,also displayed. ct of the chest with and witho...,0.0,0.543576
5684,401378,199930.0,abdomen. ct of the pelvis with and without iv ...,0.0,0.511333
5685,401379,199930.0,totally drained 3800cc. later around two thous...,0.0,0.489785


In [91]:
precision_recall_curve(y, temp)

In [ ]:
pr_thres

In [ ]:
vote_df

In [ ]:
pr_thres[pr_thres.prec > 0.799999].reset_index()

In [ ]:
temp = pr_thres[pr_thres.prec > 0.799999].reset_index()
rp80 = 0
if temp.size == 0:
    print('Test Sample too small or RP80=0')
else:
    rp80 = temp.iloc[0].recall
    print('Recall at Precision of 80 is {}', rp80)

step 3

In [73]:
train_loss=100000
number_training_steps=1
global_step_check=0

In [74]:
output_dir = '/home/haotiang/notebook'
readmission_mode = 'early'
string = 'logits_clinicalbert_'+readmission_mode+'_chunks.csv'

os.path.join(output_dir, string)

'/home/haotiang/notebook/logits_clinicalbert_early_chunks.csv'

In [92]:
df.to_csv(os.path.join(output_dir, string))

In [93]:
df_test = pd.read_csv(os.path.join(data_dir, "test.csv"))
fpr, tpr, df_out = vote_score(df_test, logits_history)

string = 'logits_clinicalbert_'+readmission_mode+'_readmissions.csv'
df_out.to_csv(os.path.join(output_dir,string))

rp80 = vote_pr_curve(df_test, logits_history)

result = {'eval_loss': eval_loss,
          'eval_accuracy': eval_accuracy,                 
          'global_step': global_step_check,
          'training loss': train_loss/number_training_steps,
          'RP80': rp80}

Recall at Precision of 80 is {} 0.5855513307984791


In [94]:
result

{'RP80': 0.5855513307984791,
 'eval_accuracy': 0.7100404431158783,
 'eval_loss': 0.5364960494651636,
 'global_step': 0,
 'training loss': 100000.0}

### end

In [ ]:
df_test

In [ ]:
logits_history